# 2015 IOU Electric Rates map using Bokeh

This notebook creates a choropleth map of investor owned electric utility rates using the bokeh package in python

first we import everything

In [1]:
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource  
from bokeh.sampledata import us_states
from bokeh.plotting import figure, show, output_file
import pandas as pd
import numpy as np
from collections import OrderedDict

# Read in Data
read rate data into dataframe from link 
create a dict of us_state data from bokeh sample (I hate this data)
attempt to sort the dict into an ordered dict so that the hovertool will match the state it is hovering over (doesn't work yet)

In [2]:
#df = pd.read_csv('http://en.openei.org/doe-opendata/dataset/011e736e-0aa0-4c73-9dd6-e943e3443d9c/resource/6cc430da-de84-4f60-a3e3-809d73a0e8e1/download/iouzipcodes2015.csv')
#for local development
df = pd.read_csv('iouzipcodes2015.csv')

df_mean = df.groupby(by=['state'])['res_rate'].mean().reset_index()
df_mean['comm_rate'] = df.groupby(by=['state'])['comm_rate'].mean().values
df_mean['ind_rate'] = df.groupby(by=['state'])['ind_rate'].mean().values
#print float(max(df_mean['res_rate']))

us_states = us_states.data.copy()
us_states_order = OrderedDict()

for k in sorted(us_states):
    us_states_order[k] = us_states.get(k)

#print us_states_order

drop HI and AK from both the electric data and the us_states data because they are really far from the continental US and will skew the map 

Also need to add a Nebraska (NE) entry for the electric data because there is no data for NE. If a NE entry isn't added, then the hovertool data for every state after Nebraska (alphabetically) will be off by one. 

create dicts for the x and y coordinates of the shapes of the states

In [3]:
df_mean = df_mean[df_mean.state != 'HI']
df_mean = df_mean[df_mean.state != 'AK']
del us_states_order["HI"]
del us_states_order["AK"]

#ADDED NE ENTRY FOR LOCAL FILE
#NEED TO ADD CODE TO ADD A NE ENTRY FROM RAW DATA
#df_mean.add('helpmeplease')
#(zip='00000',eiaid='000',utility_name='nodata',state='NE',service_type='nodata',ownership='nodata',comm_rate=0.0,ind_rate=0.0,res_rate=0.0)

state_xs = [us_states_order[code]["lons"] for code in us_states_order]
state_ys = [us_states_order[code]["lats"] for code in us_states_order]

first we define a color scale to be used in the map

Then we go through all the states in the us_states_order dict and use their residential rate
from the df_mean dataframe to determine a color index. index is determined by dividing each rate by the maximum rate, multiplying by 5 (number of colors in the scale) and rounding to the nearest integer. states with no data are made white.

In [4]:
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

state_colors = []
for state_id in us_states_order:
    try:
        #print state_id
        total = df_mean.loc[df_mean['state']==state_id]['res_rate'].values[0]
        #print total
        idx = int(round((total/float(max(df_mean['res_rate'])))*5,0))
        #print idx
        state_colors.append(colors[idx])
    except IndexError:
        state_colors.append("white")


Next we go through and define the output html file
This package is super cool because it creates an interactive html file. that way, we don't need to have server access to the location that we want to display the map... we just link to a single html file!!

In [5]:
output_file("choropleth.html", title="choropleth.py example")

This part took a while to figure out. In order to get the hovertool to work, we need to define a columndatasource with all the data that will be used. This is one of the main reasons we needed to make the NE entry in the electric data, because these columns in the columndatasource need to match up one for one otherwise they will be miss matching. Since the individual columns in the columndatasource aren't forced to be the same size you can get some really wacky hovertool output without realizing why.

In [6]:
#print df_mean
source = ColumnDataSource(data=dict(
        x=state_xs,
        y=state_ys,
        states=us_states_order.keys(),
        rates=(df_mean['res_rate']*100),
        crates=(df_mean['comm_rate']*100),
        irates=(df_mean['ind_rate']*100),
    ))


now we create the figure and define the hovertool display values

In [7]:
p = figure(title="2015 IOU Residential Electric Rates (Continental US)", toolbar_location="left",
    plot_width=1100, plot_height=700, tools='hover')
p.patches(state_xs, state_ys,source=source, fill_color=state_colors, fill_alpha=0.7,
    line_color="white", line_width=0.5)
p.patches(state_xs, state_ys,source=source, fill_alpha=0.0,
    line_color="#884444", line_width=2)

p.select_one(HoverTool).tooltips = [
    ('State', '@states'),
    ('Residential Rate', '@rates cents / kWh'),
    ('Commercial Rate', '@crates cents / kWh'),
    ('Industrial Rate', '@irates cents / kWh'),
]

turn off gridlines and axes and show the graph!!

In [8]:
p.legend.location = "bottom_left"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
#print df_mean['state'].count()
#print us_states_order.keys()
show(p)